In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

from sklearn import svm, neighbors, tree
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor # ???

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
# DATA EXTRACTION
train_df = pd.read_csv('data/TRAIN_DATA.csv', sep=';', encoding='utf_8')
final_test_df = pd.read_csv('data/TEST_INPUT.csv', sep=';', encoding='utf_8')

In [3]:
# SPLIT DATA
# Data is already sepparated between train and test sets
# but we are not provided with the testing results thus we wont use them for testing the models, only to provide final results 
# that will later be checked. Therefore, split in the train data is still needed.

features = list(train_df.columns)
target = 'WG'
features.remove(target)

X = train_df[features]
y = train_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,  
                                                    random_state=0,)

In [4]:
# PREPROCESSING
# We only work with numerical features this time

numerical_features = X_train.select_dtypes(include=['float64', 'int']).columns.to_list()
numerical_transformer = Pipeline(
    steps = [
        ("scaler", StandardScaler()),
        ("missing_values", SimpleImputer(strategy="mean")),
    ])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
],
    remainder='passthrough'
)

In [5]:
# REGRESSION MODELS WITH PIPELINE UNION
SV = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", svm.SVR())]
)

Kneighbors = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", neighbors.KNeighborsRegressor())]
)

Tree = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", tree.DecisionTreeRegressor())]
)


In [6]:
# PARAM GRID FOR TWO HYPERPARAMETERS

SV_param_grid = {
    'regressor__kernel' : ['linear', 'rbf', 'sigmoid'],
    'regressor__C' : [1,5,10],
    'regressor__gamma': [0.001, 0.01, 1]
}

KN_param_grid = {
    'regressor__n_neighbors': [2,3,5,7,10],
}

Tree_param_grid = {
    'regressor__splitter': ['best','random'],
    'regressor__max_depth': [3,5,7,8],
    'regressor__min_samples_leaf': [1,5,10]
}

In [7]:
# CREATTING DE GRIDS FOR EVERY MODEL

SV_search = GridSearchCV(estimator = SV, 
                      param_grid = SV_param_grid, 
                      n_jobs=1, 
                      verbose=4,
                      scoring = "roc_auc",
                      cv = 3,
                      return_train_score=True)
KN_search = GridSearchCV(Kneighbors, 
                      KN_param_grid, 
                      n_jobs=1, 
                      verbose=4,
                      scoring = "roc_auc",
                      cv = 3,
                      return_train_score=True)
Tree_search = GridSearchCV(Tree, 
                      Tree_param_grid, 
                      n_jobs=1, 
                      verbose=4,
                      scoring = "roc_auc",
                      cv = 3,
                      return_train_score=True)

In [8]:
# TRAINING THE MODELS

SV_search.fit(X_train,y_train)
KN_search.fit(X_train,y_train)
Tree_search.fit(X_train,y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=linear;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=linear;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=linear;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__C=1, regressor__gamma=0.001, regressor__kernel=sigmoid;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__C=1, regressor__gamma=0.001, regre

[CV 3/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=linear;, score=(train=nan, test=nan) total time=   0.1s
[CV 1/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=rbf;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=sigmoid;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=sigmoid;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__C=10, regressor__gamma=0.01, regressor__kernel=sigmoid;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__C=10, regressor__gamma=1, regressor__kernel=linear;, score=(train=nan, test=nan) total time= 

[CV 2/3] END regressor__max_depth=7, regressor__min_samples_leaf=1, regressor__splitter=best;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__max_depth=7, regressor__min_samples_leaf=1, regressor__splitter=best;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__max_depth=7, regressor__min_samples_leaf=1, regressor__splitter=random;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__max_depth=7, regressor__min_samples_leaf=1, regressor__splitter=random;, score=(train=nan, test=nan) total time=   0.0s
[CV 3/3] END regressor__max_depth=7, regressor__min_samples_leaf=1, regressor__splitter=random;, score=(train=nan, test=nan) total time=   0.0s
[CV 1/3] END regressor__max_depth=7, regressor__min_samples_leaf=5, regressor__splitter=best;, score=(train=nan, test=nan) total time=   0.0s
[CV 2/3] END regressor__max_depth=7, regressor__min_samples_leaf=5, regressor__splitter=best;, score=(train=nan, test=nan) total time=   0.0s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler()),
                                                                                         ('missing_values',
                                                                                          SimpleImputer())]),
                                                                         ['Hour',
                                                                          'TL1H80',
                                                                          'TL2H80',
                                                                          'TL3H80',
                                                                          'TL4H80',
                                                                          'TL5H80',
                                                                          'TL6H80',
                                                                          'TL7H80',
                                                                          'TL8H80',
                                                                          'TL9H80',
                                                                          'TL10H80',
                                                                          'WSL1H80',
                                                                          'WSL2H80',
                                                                          'WSL3H80',
                                                                          'WSL4H80',
                                                                          'WSL5H8...
                                                                          'WSL8H80',
                                                                          'WSL9H80',
                                                                          'WSL10H80',
                                                                          'WDL1H80',
                                                                          'WDL2H80',
                                                                          'WDL3H80',
                                                                          'WDL4H80',
                                                                          'WDL5H80',
                                                                          'WDL6H80',
                                                                          'WDL7H80',
                                                                          'WDL8H80',
                                                                          'WDL9H80', ...])])),
                                       ('regressor', DecisionTreeRegressor())]),
             n_jobs=1,
             param_grid={'regressor__max_depth': [3, 5, 7, 8],
                         'regressor__min_samples_leaf': [1, 5, 10],
                         'regressor__splitter': ['best', 'random']},
             return_train_score=True, scoring='roc_auc', verbose=4)

In [18]:
# BEST MODELS

print('SV:',SV_search.best_params_,'(best)\n',
      SV_search.best_estimator_.score(X_train, y_train),'(train score)\n',
      SV_search.best_estimator_.score(X_test, y_test), '(test score)\n')
print('KNN:',KN_search.best_params_, '(best)\n',
      KN_search.best_estimator_.score(X_train, y_train),'(train score)\n',
      KN_search.best_estimator_.score(X_test, y_test), '(test score)\n',)
print('Tree:',Tree_search.best_params_, '(best)\n',
      Tree_search.best_estimator_.score(X_train, y_train), '(train score)\n',
      Tree_search.best_estimator_.score(X_test, y_test), '(test score)\n',)

SV: {'regressor__C': 1, 'regressor__gamma': 0.001, 'regressor__kernel': 'linear'} (best)
 0.8669793890340167 (train score)
 0.881711326253592 (test score)

KNN: {'regressor__n_neighbors': 2} (best)
 0.9832214990775999 (train score)
 0.950945674867764 (test score)

Tree: {'regressor__max_depth': 3, 'regressor__min_samples_leaf': 1, 'regressor__splitter': 'best'} (best)
 0.8545907205022312 (train score)
 0.8498224426478997 (test score)



In [25]:
def root_mean_squared_error(pred, true) -> float:
    
    return np.sqrt(mean_squared_error(true,pred))

In [28]:
# MEASSURING PERFORMANCE

# RMSE

SV_RMSE = root_mean_squared_error(SV_search.predict(X_test), y_test)
print('RMSE SVR-model:', SV_RMSE)

KN_RMSE = root_mean_squared_error(KN_search.predict(X_test), y_test)
print('RMSE KNN-model:', KN_RMSE)

Tree_RMSE = root_mean_squared_error(Tree_search.predict(X_test), y_test)
print('RMSE Tree-model:', Tree_RMSE)

RMSE SVR-model: 20.445340387607175
RMSE KNN-model: 13.166229371402794
RMSE Tree-model: 23.03696604730559
